In [36]:
# %load ../../../preconfig.py
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
#sns.set(font='SimHei')
#plt.rcParams['axes.grid'] = False

import numpy as np

import pandas as pd
pd.options.display.max_rows = 20

#import sklearn

#import itertools

import logging
logger = logging.getLogger()

决策树简介和 Python 实现
======================
参考：

+ [Building a decision tree from scratch - a beginner tutorial](http://www.patricklamle.com/Tutorials/Decision%20tree%20python/tuto_decision%20tree.html)

+ 9.2 Tree-Based Methods - The Elements of Statistical Learning

+ [Classification and Regression Trees (CART) Theory and Applications](http://edoc.hu-berlin.de/master/timofeev-roman-2004-12-20/PDF/timofeev.pdf)

#### 0. 基本介绍
本文主要是参照 Tree-Based Methods - The Elements of Statistical Learning 来实现一个简化版范例，其算法是 CART。

决策树的思想本身非常朴素，关于它的基本介绍在网上已经非常丰富，比如：

+ [算法杂货铺——分类算法之决策树(Decision tree)](http://www.cnblogs.com/leoo2sk/archive/2010/09/19/decision-tree.html)

其主要问题是在每次决策时找到一个分割点，让生成的子集尽可能地纯净。这里涉及到四个问题:

1. 如何分割样本？

2. 如何评价子集的纯净度？

3. 如何找到单个最佳的分割点，其子集最为纯净？

4. 如何找到最佳的分割点序列，其最终分割子集总体最为纯净？

接下来，围绕上述问题，一一概要说明，并加以演示。

#### 加载数据

古话说，「三军未动，粮草先行」。

我们先加载演示数据，使用的是 sklearn 自带的测试用例。

In [2]:
from sklearn.datasets import load_iris
data = load_iris()

In [31]:
# 准备特征数据
X = pd.DataFrame(data.data, 
                 columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])
X.head(2)

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


In [4]:
# 准备标签数据
y = pd.DataFrame(data.target, columns=['target'])
y.replace(to_replace=range(3), value=data.target_names, inplace=True)
y.head(3)

,target
0,setosa
1,setosa
2,setosa


In [5]:
# 组建样本 [特征，标签]
samples = pd.concat([X, y], axis=1) #, keys=["x", "y"])
samples.head(3)

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


#### 1.0 如何分割样本
决策树的分割方法是取一个特征 $f$ 和阈值 $t$，以此为界将样本 $X$ 拆分为两个子集 $X_l, X_r$。其数学表达形同：

\begin{align}
    X = \begin{cases}
        X_l, \ \text{if } X[f] < t \\
        X_r, \ \text{if } X[f] \geq t
    \end{cases}
\end{align}

In [6]:
def splitter(samples, feature, threshold):
    # 按特征 f 和阈值 t 分割样本
    
    left_nodes = samples.query("{f} < {t}".format(f=feature, t=threshold))
    right_nodes = samples.query("{f} >= {t}".format(f=feature, t=threshold))
    
    return {"left_nodes": left_nodes, "right_nodes": right_nodes}

In [32]:
split = splitter(samples, "sepal_length", 5)

# 左子集
x_l = split["left_nodes"].loc[:, "target"].value_counts()
x_l

setosa        20
versicolor     1
virginica      1
Name: target, dtype: int64

In [33]:
# 右子集
x_r = split["right_nodes"].loc[:, "target"].value_counts()
x_r

virginica     49
versicolor    49
setosa        30
Name: target, dtype: int64

#### 2. 如何评价子集的纯净度？

从常理来说，我们希望分割子集尽可能地纯净，最好是单个子集就只含有一类标签，从而保证决策结果精准。

那么什么样的评价函数，可以用来度量各子集的纯净度呢？

以刚才计算结果为例， $x_l$ 主要标签是 setosa，非常纯净，而 $x_r$ 则三种标签势均力敌，非常混杂。所以思路是，若一种标签在子集中占比非常大，则此子集就较纯净；若各标签占比差别不大，就较为混杂。

常用的评价函数正是计算各标签 $c_k$ 在子集中的占比 $p_k = c_k / \sum (c_k)$，并通过组合 $p_k$ 来描述占比集中或分散。

In [9]:
def calc_class_proportion(node):
    # 计算各标签在集合中的占比
    
    y = node["target"]
    return y.value_counts() / y.count()

In [34]:
calc_class_proportion(split["left_nodes"])

setosa        0.909091
versicolor    0.045455
virginica     0.045455
Name: target, dtype: float64

In [10]:
calc_class_proportion(split["right_nodes"])

virginica     0.382812
versicolor    0.382812
setosa        0.234375
Name: target, dtype: float64

主要的评价函数有三种，它们评价的是集合的不纯度（值越大，集合越混杂）。

先做些数学定义以便于描述：     
假设对于集合 $m$ 有 $N_m$ 个样本，可分割成 $R_m$ 子集。     
若总的标签类别有 $K$ 种，则标签 $k$ 在此集合中的占比为：
\begin{equation}
    \hat{p}_{m k} = \frac{1}{N_m} \displaystyle \sum_{x_i \in R_m} I(y_i = k)
\end{equation}

且令标签 $k$ 是占比最大的标签，即 $k(m) = \operatorname{arg max}_k \hat{p}_{m k}$.

##### 1. Misclassification error
我们一般把集合的分类结果定义为占比最大的标签，那么落在此集合中的其它标签就是误分类。其比率是 $1 - \hat{p}_{m k}(m)$.

In [11]:
def misclassification_error(node):
    p_mk = calc_class_proportion(node)
    
    return 1 - p_mk.max()

In [12]:
misclassification_error(split["left_nodes"])

0.090909090909090939

In [13]:
misclassification_error(split["right_nodes"])

0.6171875

对于二分类问题，误分类率和占比 $p$ 的关系可划图为：

In [ ]:
binary_p = pd.

In [14]:
def gini_index(node):
    p_mk = calc_class_proportion(node)
    
    return (p_mk * (1 - p_mk)).sum()

In [15]:
gini_index(split["left_nodes"])

0.1694214876033058

In [16]:
gini_index(split["right_nodes"])

0.6519775390625

In [17]:
def cross_entropy(node):
    p_mk = calc_class_proportion(node)
    
    return - (p_mk * p_mk.apply(np.log)).sum()

In [18]:
cross_entropy(split["left_nodes"])

0.36764947740014225

In [19]:
cross_entropy(split["right_nodes"])

1.075199711851601

#### 最优分割

In [20]:
def calc_impurity_measure(node, feathure, threshold, measure, min_nodes=5):
    child = splitter(node, feathure, threshold)
    left = child["left_nodes"]
    right = child["right_nodes"]
    
    if left.shape[0] <= min_nodes or right.shape[0] <= min_nodes:
        return 0
    
    
    impurity = pd.DataFrame([], 
                       columns=["score", "rate"],
                       index=[])
    
    impurity.loc["all"] = [measure(node), node.shape[0]]
    impurity.loc["left"] = [-measure(left), left.shape[0]]
    impurity.loc["right"] = [-measure(right), right.shape[0]]
    
    impurity["rate"] /= impurity.at["all", "rate"]
    
    logger.info(impurity)
    
    return (impurity["score"] * impurity["rate"]).sum()

In [21]:
calc_impurity_measure(samples, "sepal_length", 5, gini_index)

0.08546401515151514

In [22]:
def find_best_threshold(node, feature, measure):
    threshold_candidates = node[feature].quantile(np.arange(0, 1, 0.2))
    
    res = pd.Series([], name=feature)
    for t in threshold_candidates:
        res[t] = calc_impurity_measure(node, feature, t, measure)
    
    logger.info(res)
    
    if res.max() == 0:
        return None
    else:
        return res.argmax()

In [23]:
find_best_threshold(samples, "sepal_width", gini_index)

3.3999999999999999

In [24]:
find_best_threshold(samples, "sepal_length", gini_index)

5.5999999999999996

In [25]:
def find_best_split(node, measure):
    if node["target"].unique().shape[0] <= 1:
        return None
    
    purity_gain = pd.Series([], name="feature")
    
    for f in node.drop("target", axis=1).columns:
        purity_gain[f] = find_best_threshold(node, f, measure)
        
    if pd.isnull(purity_gain.max()):
        return None
    else:
        best_split = {"feature": purity_gain.argmax(), "threshold": purity_gain.max()}
        best_split["child"] = splitter(node, **best_split)

        return best_split

In [26]:
find_best_split(samples, gini_index)

{'child': {'left_nodes':      sepal_length  sepal_width  petal_length  petal_width      target
  0             5.1          3.5           1.4          0.2      setosa
  1             4.9          3.0           1.4          0.2      setosa
  2             4.7          3.2           1.3          0.2      setosa
  3             4.6          3.1           1.5          0.2      setosa
  4             5.0          3.6           1.4          0.2      setosa
  5             5.4          3.9           1.7          0.4      setosa
  6             4.6          3.4           1.4          0.3      setosa
  7             5.0          3.4           1.5          0.2      setosa
  8             4.4          2.9           1.4          0.2      setosa
  9             4.9          3.1           1.5          0.1      setosa
  ..            ...          ...           ...          ...         ...
  59            5.2          2.7           3.9          1.4  versicolor
  60            5.0          2.0         

#### 递归查最优分割点

In [27]:
class BinaryNode:
    def __init__(self, samples, max_depth, measure=gini_index):
        self.samples = samples
        self.max_depth = max_depth
        self.measure = measure
        
        self.is_leaf = False
        self.class_ = None
        
        self.left = None
        self.right = None
        
        self.best_split = None
    
    def split(self, depth):
        if depth > self.max_depth:
            self.is_leaf = True
            self.class_ = self.samples["target"].value_counts().argmax()
            return
        
        best_split = find_best_split(self.samples, self.measure)
        if pd.isnull(best_split):
            self.is_leaf = True
            self.class_ = self.samples["target"].value_counts().argmax()
            return

        self.best_split = best_split
        left = self.best_split["child"]["left_nodes"]
        self.left = BinaryNode(left.drop(best_split["feature"], axis=1), self.max_depth)
        
        right = self.best_split["child"]["right_nodes"]
        self.right = BinaryNode(right.drop(best_split["feature"], axis=1), self.max_depth)

        self.left.split(depth+1)
        self.right.split(depth+1)

In [28]:
binaryNode = BinaryNode(samples, 3)
binaryNode.split(0)

In [29]:
def show(node, depth):
    if node.left:
        show(node.left, depth+1)
    
    if node.is_leaf:
        print("{}{}".format("\t"*(depth+2), node.class_))
        return 
    else:
        print("{}{}: {}".format("\t"*depth,
                                node.best_split["feature"],
                                node.best_split["threshold"]))
    if node.right:
        show(node.right, depth+1)

In [30]:
show(binaryNode, 0)

				versicolor
	sepal_width: 2.8200000000000003
					setosa
		petal_length: 1.6
						setosa
			petal_width: 0.4
						setosa
sepal_length: 5.6
					versicolor
		sepal_width: 3.1
					versicolor
	petal_length: 4.8
						versicolor
			petal_width: 1.8
						virginica
		sepal_width: 2.9
						virginica
			petal_width: 2.0
						virginica
